In [1]:
import matplotlib
matplotlib.use('Agg')
from scipy.spatial import distance
import pandas as pd
import argparse,sys,os
import mplhep as hep
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import numpy as np
from sklearn.metrics import auc
from matplotlib import rcParams
from scipy import integrate
import tqdm
import scipy.ndimage as sc
import pyarrow.parquet as pq

plt.style.use([hep.style.ROOT, hep.style.firamath])

opath = "./12Jun23_QCD_v2/results/analyze_parquet80"
year = "2017"
parquet = ["./../12Jun23_QCD_v2/results/signal_region1.parquet.gzip","./../12Jun23_QCD_v2/results/signal_region2.parquet.gzip"]
os.system(f"mkdir -p {opath}")
os.system(f"mkdir -p {opath}/distributions/")
rlabel = f"{year} (13 TeV)"



/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.6.9' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
_ptlow = 200
_pthigh = 1200
_msdlow = 40
_msdhigh = 400
_rholow = -7
_rhohigh = -1.5
_nbins = 81


In [3]:
def axis_settings(ax):
    import matplotlib.ticker as plticker
    #ax.xaxis.set_major_locator(plticker.MultipleLocator(base=20))
    ax.xaxis.set_minor_locator(plticker.AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(plticker.AutoMinorLocator(5))
    ax.tick_params(direction='in', axis='both', which='major', labelsize=24, length=12)#, labelleft=False )
    ax.tick_params(direction='in', axis='both', which='minor' , length=6)
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')    
    #ax.grid(which='minor', alpha=0.5, axis='y', linestyle='dotted')
    ax.grid(which='major', alpha=0.9, linestyle='dotted')
    return ax

shortname = { "ZJetsToQQ" : "Z", "QCD" : "QCD" }


In [4]:
inlay_font = {
        #'fontfamily' : 'arial',
        #'weight' : 'normal',
        'size'   : 14
}
axis_font = {
        #'fontfamily' : 'arial',
        #'weight' : 'normal',
        'size'   : 24
}
legend_font = {
        #'family' : 'sans-serif',
        #'weight' : 'normal',
        'size'   : 20
}
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=18)
plt.rc('axes', titlesize=18)
plt.rc('legend', fontsize=10)

In [5]:
master_data = None
if type(parquet) == list:
    for p in parquet[:]:
        parquet_file = pq.ParquetFile(p,memory_map=True)
        if master_data is None:
            master_data = parquet_file.read().to_pandas()
        else:
            master_data = pd.concat((master_data,parquet_file.read().to_pandas()))
            


In [6]:
print(master_data.memory_usage().sum()/1e9)

3.209710061


In [7]:
master_data.shape, master_data.columns

((60560565, 9),
 Index(['msd', 'msd_corrected', 'n2b1', 'n2b1_ddt', 'n2b1_ddt_smoothed', 'pt',
        'rho', 'weight', 'process'],
       dtype='object'))

In [8]:
def find_pctl(score,weights,ibin,pctl=0.05,reverse=False,):
    pdf,bins = np.histogram(score, bins=np.linspace(0,0.5,40), weights=weights,density=True,)
    fig,ax = plt.subplots()
    ax = axis_settings(ax)
    cdf = np.cumsum(pdf)*np.diff(bins)
    pctl_bin = np.searchsorted(cdf, [pctl if reverse else 1.-pctl])
    ax.stairs(pdf, bins)
    ax.set_xlabel(r"Jet $N_2$")
    ax.set_ylabel(r"Normalized events")
    ax.axvline(bins[pctl_bin[0]],linestyle="--",color="red",lw=2,)
    plt.savefig(opath+f"/distributions/rho_pt_{ibin}.png")
    plt.savefig(opath+f"/distributions/rho_pt_{ibin}.pdf")
    return bins[pctl_bin[0]]


In [9]:
def make_ddt_map(qcdname, score, msd, pt, weights, taggername, reverse=False):

    rho = 2*np.log(msd/pt)

    plt.clf()
    fig,ax=plt.subplots()
    ax = axis_settings(ax)
    hep.cms.label("Preliminary",rlabel=rlabel, data=False)
    ax.hist(pt,bins=np.linspace(_ptlow,_pthigh,_nbins),weights=weights,histtype="step",lw=2,label="QCD")
    ax.set_xlabel("Jet $p_{T}$ (GeV)")   
    ax.set_ylabel(r"Events")
    ax.set_yscale("log")
    ax.legend()
    plt.tight_layout()
    plt.savefig(opath+f"/pt.png")
    plt.savefig(opath+f"/pt.pdf")

      
    plt.clf()
    fig,ax=plt.subplots()
    ax = axis_settings(ax)
    hep.cms.label("Preliminary",rlabel=rlabel, data=False)
    ax.hist(rho,bins=np.linspace(_rholow,_rhohigh,_nbins),weights=weights,histtype="step",lw=2,label="QCD")
    ax.set_xlabel(r"Jet $\rho$ (GeV)")   
    ax.set_ylabel(r"Events")
    ax.set_yscale("log")
    ax.legend()
    plt.tight_layout()
    plt.savefig(opath+f"/rho.png")
    plt.savefig(opath+f"/rho.pdf")


    plt.clf()
    fig,ax=plt.subplots()
    ax = axis_settings(ax)

    hep.cms.label("Preliminary",rlabel=rlabel, data=False)
    h2, rhoedges, ptedges, im = plt.hist2d(rho, pt,
                                         bins=[np.linspace(_rholow, _rhohigh, _nbins), np.linspace(_ptlow,_pthigh,_nbins)] ,
                                         weights=weights, density=False, 
                                         norm=matplotlib.colors.LogNorm(),
                                         )
    ax.set_xlim(_rholow, _rhohigh)
    ax.set_ylim(_ptlow,_pthigh)
    ax.set_xlabel(r"Jet $\rho$")
    ax.set_ylabel("Jet $p_{T}$ (GeV)")   
    ax.set_aspect(abs(_rhohigh-_rholow)/(_pthigh-_ptlow))
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(opath+f"/rho_pt.png") 
    plt.savefig(opath+f"/rho_pt.pdf")

    #sys.exit()
    ddt_map = np.zeros(shape=(h2.shape[0],h2.shape[1]))
    counter = 0
    for irho in tqdm.tqdm(range(len(rhoedges)-1)):
        for ipt in range(len(ptedges)-1):
            sel = (rho <= rhoedges[irho+1]) & (rho > rhoedges[irho]) & (pt <= ptedges[ipt+1]) & (pt > ptedges[ipt])
            score_tmp,weights_tmp = score[sel], weights[sel]
            #print(score_tmp,weights_tmp)
            ddt_map[irho, ipt] = find_pctl(score_tmp,weights_tmp,"pt_{}_{}_rho_{}_{}".format(round(rhoedges[irho],2),round(rhoedges[irho+1],2),round(ptedges[ipt],2),round(ptedges[ipt+1],2)),reverse=reverse)
            counter += 1 
    plt.clf()
    fig,ax = plt.subplots()
    ax = axis_settings(ax)
    plt.imshow(ddt_map.T,origin="lower",extent=[_rholow,_rhohigh,_ptlow,_pthigh,],aspect=abs(_rhohigh-_rholow)/(_pthigh-_ptlow),interpolation='none',)
    ax.set_xlabel(r"Jet $\rho$")
    ax.set_ylabel("Jet $p_{T}$ (GeV)")
    hep.cms.label("Preliminary",rlabel=rlabel, data=False)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(opath+f"/{taggername}_ddtmap_rho_pt.png")
    plt.savefig(opath+f"/{taggername}_ddtmap_rho_pt.pdf")

    fig,ax = plt.subplots()
    ax = axis_settings(ax)
    ddt_map_smoothed = sc.gaussian_filter(ddt_map.T,1)
    np.savez(opath+f"/ddt_map_smoothed",ddt_map=ddt_map_smoothed,rhoedges=rhoedges,ptedges=ptedges)
    plt.imshow(ddt_map_smoothed,origin="lower",extent=[_rholow,_rhohigh,_ptlow,_pthigh,],aspect=abs(_rhohigh-_rholow)/(_pthigh-_ptlow),interpolation='none',)
    ax.set_xlabel(r"Jet $\rho$")
    ax.set_ylabel("Jet $p_{T}$ (GeV)")
    hep.cms.label("Preliminary",rlabel=rlabel, data=False)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.savefig(opath+f"/{taggername}_ddtmap_rho_pt_smoothed.png")
    plt.savefig(opath+f"/{taggername}_ddtmap_rho_pt_smoothed.pdf")
 
    import hist
    import json
    import correctionlib.convert
    h = (
        hist.Hist.new
        .Reg(_nbins-1, _rholow, _rhohigh, name="rho")
        .Reg(_nbins-1, _ptlow, _pthigh, name="pt")
    )
    sfhist = hist.Hist(*h.axes[:], data=ddt_map)
    plt.clf()
    fig,ax=plt.subplots()
    sfhist.plot2d()
    plt.savefig(opath+f"/{taggername}_ddtmap_rho_pt_closure.pdf")
    plt.clf()


    sfhist.name = "ddtmap_5pct_n2"
    sfhist.label = "out"

    

    sfhist_correctionlib = correctionlib.convert.from_histogram(sfhist)
    sfhist_correctionlib.data.flow = "clamp"
    cset = correctionlib.schemav2.CorrectionSet(
      schema_version=2,
      description="my N2DDT correction",
      corrections=[
        sfhist_correctionlib,
      ],
    )
    with open(opath+f"/{taggername}_ddtmap_rho_pt.json","w") as outfile:
        outfile.write(cset.json(exclude_unset=True))

    sfhist_smoothed = hist.Hist(*h.axes[:], data=ddt_map_smoothed)
    sfhist_smoothed.name = "ddtmap_5pct_n2_smoothed"
    sfhist_smoothed.label = "out"
    sfhist_correctionlib = correctionlib.convert.from_histogram(sfhist_smoothed)
    sfhist_correctionlib.data.flow = "clamp"
    cset = correctionlib.schemav2.CorrectionSet(
      schema_version=2,
      description="my N2DDT correction",
      corrections=[
        sfhist_correctionlib,
      ],
    ) 
    with open(opath+f"/{taggername}_ddtmap_rho_pt_smoothed.json","w") as outfile:
        outfile.write(cset.json(exclude_unset=True))
    return


In [ ]:
make_ddt_map("QCD",master_data["n2b1"],master_data["msd_corrected"],master_data["pt"],master_data["weight"],"n2b1",True)


/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
  0%|                                                                                                                                                    | 0/80 [00:00<?, ?it/s]/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.su

  1%|█▋                                                                                                                                       | 1/80 [01:58<2:36:07, 118.58s/it]/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.su

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.su

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.su

...

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.su

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.su

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
 18%|███████████████████████▊                                                                                                                | 14/80 [25:43<2:03:34, 112.34s/it]/afs/cern.ch

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
 21%|████████████████████████████▉                                                                                                           | 17/80 [31:35<2:01:19, 115.54s/it]/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch

/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
/afs/cern.ch/work/j/jekrupa/public/bamboodev/bamboovenv/lib/python3.9/site-packages/numpy/lib/histograms.py:906: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges
 26%|███████████████████████████████████▋                                                                                                    | 21/80 [39:22<1:54:58, 116.93s/it]/afs/cern.ch